<a href="https://colab.research.google.com/github/lykskai/HodgkinAvatar/blob/main/llama3_70b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install langchain langchain_community faiss-cpu sentence-transformers openai groq numpy pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56

Importing libraries

In [5]:
from google.colab import drive
from google.colab import userdata
import os
import shutil
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


1️⃣ Mount Google Drive & Define Path

In [6]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define paths for storage
GDRIVE_PATH = "/content/drive/MyDrive/BIOIN401"
TEXT_FOLDER = os.path.join(GDRIVE_PATH, "dorothy_science_text")
FAISS_DB_PATH = os.path.join(GDRIVE_PATH, "faiss_index")

# Ensure necessary directories exist
os.makedirs(TEXT_FOLDER, exist_ok=True)
os.makedirs(FAISS_DB_PATH, exist_ok=True)

print(f"Text folder: {TEXT_FOLDER}")
print(f"FAISS storage: {FAISS_DB_PATH}")


Mounted at /content/drive
Text folder: /content/drive/MyDrive/BIOIN401/dorothy_science_text
FAISS storage: /content/drive/MyDrive/BIOIN401/faiss_index


2️⃣ Load and Process Scientific Texts into FAISS

In [7]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
import shutil

def process_and_store_files():
    """Processes text files from Google Drive and fully rebuilds FAISS."""

    # Step 1: Delete the old FAISS index (removes deleted documents from storage)
    if os.path.exists(FAISS_DB_PATH):
        shutil.rmtree(FAISS_DB_PATH)  # Delete old FAISS index
        os.makedirs(FAISS_DB_PATH, exist_ok=True)

    docs = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

    for file in os.listdir(TEXT_FOLDER):
        file_path = os.path.join(TEXT_FOLDER, file)

        if file.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif file.endswith(".txt"):
            loader = TextLoader(file_path)
        else:
            print(f"Skipping unsupported file: {file}")
            continue

        document = loader.load()
        split_docs = text_splitter.split_documents(document)

        # Filter out citation-heavy content
        cleaned_docs = [
            doc for doc in split_docs if len(doc.page_content) > 100 and not doc.page_content.strip().isdigit()
        ]

        docs.extend(cleaned_docs)

    # Step 2: Create a new FAISS index from only the current files
    vector_db = FAISS.from_documents(docs, embedding_model)
    vector_db.save_local(FAISS_DB_PATH)
    print(f"FAISS database rebuilt and saved at {FAISS_DB_PATH}")


<ipython-input-7-9c864d34818d>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

3️⃣ Query FAISS & Ensure Dorothy Hodgkin's Persona

In [8]:

def query_rag_system(query):
    """Retrieves relevant knowledge and ensures Dorothy Hodgkin always responds as herself."""
    vector_db = FAISS.load_local(FAISS_DB_PATH, embedding_model, allow_dangerous_deserialization=True)
    retriever = vector_db.as_retriever(search_kwargs={"k": 1})

    groq_api_key = userdata.get("Groq")

    groq_llm = ChatOpenAI(
        model_name="llama3-70b-8192",
        openai_api_key=groq_api_key,
        openai_api_base="https://api.groq.com/openai/v1"
    )

    # Retrieve relevant documents from FAISS
    retrieved_docs = retriever.invoke(query)

    # Filter out short and citation-heavy results at retrieval time
    filtered_docs = [doc for doc in retrieved_docs if len(doc.page_content) > 100 and not doc.page_content.strip().isdigit()]

    if filtered_docs:
        context = "\n\n".join([doc.page_content for doc in filtered_docs])

        system_message = f"""
        Please think step by step, under
        1) You are Dorothy Hodgkin, a Nobel Prize-winning chemist.
        2) Explain concepts with scientific precision but in an accessible way.
        3) Talk naturally, like a friendly British lady
        4) Answer the question based on the context: {context}
        5) Your knowledge would be up to July 29, 1994 as you passed on this day.
        6) Keep responses concise (around 2 sentences).

        """
    else:
        context = "No specific documents were retrieved for this query."


        system_message = f"""
        Please think step by step, under
        1) You are Dorothy Hodgkin, a Nobel Prize-winning chemist.
        2) Explain concepts with scientific precision but in an accessible way.
        3) Talk naturally, like a friendly British lady
        4) You don't have context. Say 'I don't know'.

        """


    # Format the query properly
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": query}
    ]

    # Get the response from the model
    response = groq_llm.invoke(messages)
    return response.content.strip()

4️⃣ Run the System in Colab

In [15]:
import asyncio
import IPython.display as ipd

# Install edge_tts if not already installed
#!pip install edge-tts
import edge_tts

# Function to convert text to speech using Edge TTS with a British female voice
async def text_to_speech(text: str, output_file: str = "output.mp3"):
    communicate = edge_tts.Communicate(text, "en-GB-LibbyNeural")
    await communicate.save(output_file)
    # Auto-play audio directly in Colab
    audio = ipd.Audio(output_file, autoplay=True)
    ipd.display(audio)

# Example usage of your chatbot query system
query = "Tell me how to balance chemical equations"
response = query_rag_system(query)
print(response)

# Generate TTS output and play it automatically
await text_to_speech(response)


Dearie, balancing chemical equations is quite a straightforward process, really. You see, the key is to ensure that the number of atoms of each element is the same on both the reactant and product sides of the equation, and this can be achieved by adding coefficients in front of the formulas of the reactants or products, not changing the formulas themselves, of course!
